In [1]:
import numpy as np
import warnings
from tqdm import tqdm
import shap
import pandas as pd
import joblib
import shap

X_train = pd.read_csv("/ml_data/X_train.csv").drop(columns = "Unnamed: 0")
y_train = pd.read_csv("/ml_data/y_train.csv").drop(columns = "Unnamed: 0")
X_test = pd.read_csv("/ml_data/X_test.csv").drop(columns = "Unnamed: 0")
y_test = pd.read_csv("/ml_data/y_test.csv").drop(columns = "Unnamed: 0")
feature_names = list(X_train.columns)
model = joblib.load("/artifact/mlruns/813623427044464195/69402f2e004a4032b669f7800a476315/artifacts/model/model.pkl")
# model = joblib.load("/artifact/mlruns/941649382202349625/e311be07056d4a4fa7413ec46830e13b/artifacts/model/model.pkl")
task = "regression"
target = "price"
pipeline_name = "non_remove_outlier median StandardScaler"

all_columns = []
cat_feature = []
for i in range(len(model["preprocessor"].transformers_)):
    if model["preprocessor"].transformers_[i][0] == "num":
        all_columns.extend(model["preprocessor"].transformers[i][2])
    else:
        for j in range(len(model['preprocessor'].transformers_[1][1].categories_)):
            all_columns.extend(list(model['preprocessor'].transformers_[1][1].categories_[j]))
            cat_feature.append(list(model['preprocessor'].transformers_[1][1].categories_[j]))
            
def save_to_csv():
    explainer = shap.TreeExplainer(model[1] ,feature_names = all_columns)
    X_test_ja = pd.DataFrame(model[0].fit_transform(X_test), columns = all_columns)
    ## inverse standard scaler => check_columns_that_numeric_and_inverse_it 
    X_test_ja.iloc[:,:79] = model[0].transformers_[0][1].steps[1][1].inverse_transform(X_test_ja.iloc[:,:79])
    shap_values = explainer.shap_values(model[0].fit_transform(X_test))
    global_values = pd.DataFrame(np.reshape(sum(np.abs(shap_values))/len(sum(np.abs(shap_values))), newshape = (1,-1)), columns = all_columns)
    train_dependency_values = pd.DataFrame(np.abs(shap_values), columns = ["importance_values_" + i for i in all_columns])
    pd.concat([train_dependency_values, X_test_ja], axis = 1).to_csv("/model_interpret/local.csv")
    global_values.to_csv("/model_interpret/global.csv")


In [2]:
model[0].transformers_[0][1].steps[1][1].inverse_transform(X_test_ja.iloc[:,:79])

NameError: name 'X_test_ja' is not defined

In [80]:
explainer = shap.TreeExplainer(model[1] ,feature_names = all_columns)
X_test_ja = pd.DataFrame(model[0].fit_transform(X_test), columns = all_columns)
X_test_ja.iloc[:,:79] = model[0].transformers_[0][1].steps[1][1].inverse_transform(pd.DataFrame(model[0].fit_transform(X_test_ja), columns = all_columns).iloc[:,:79])
shap_values = explainer.shap_values(model[0].fit_transform(X_test))
global_values = pd.DataFrame(np.reshape(sum(np.abs(shap_values))/len(sum(np.abs(shap_values))), newshape = (1,-1)), columns = all_columns)
train_dependency_values = pd.DataFrame(np.abs(shap_values), columns = ["importance_values_" + i for i in all_columns])

ValueError: A given column is not a column of the dataframe

In [3]:
model['preprocessor'].transformers_[1][1]

OneHotEncoder(handle_unknown='ignore')

In [81]:
model[0].transformers_[1][1].transform(X_test)

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- APL_distance
- Air-Conditioning
- Audio System
- BBQ pits
- BRT_distance
- ...
Feature names must be in the same order as they were in fit.



ValueError: X has 81 features, but OneHotEncoder is expecting 2 features as input.

In [82]:
model[0].transformers_[1][1]

OneHotEncoder(handle_unknown='ignore')

In [83]:
save_to_csv()

In [67]:
X_test.iloc[:,:79] = model[0].transformers_[0][1].steps[1][1].inverse_transform(pd.DataFrame(model[0].fit_transform(X_test), columns = all_columns).iloc[:,:79])

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`


In [68]:
X_test

,bedroom,bathroom,sqm,developer,built_year,district,luxuary,public,BRT_distance,BTS_distance,...,car park,cook,fitness,furniture,nan,pool,sauna,security,tv,view
0,0.0,1.0,30.0,2013.0,0.0,0.0,10000.0,10000.0,580.0,10000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,2.0,1.0,39.0,2014.0,0.0,0.0,10000.0,10000.0,230.0,10000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,1.0,1.0,52.0,2016.0,0.0,0.0,10000.0,10000.0,10000.0,10000.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0,1
3,1.0,1.0,35.0,2009.0,0.0,1.0,10000.0,300.0,10000.0,10000.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0,0
4,3.0,3.0,277.0,2014.0,3.0,0.0,10000.0,10000.0,540.0,10000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,1.0,1.0,65.0,2009.0,1.0,0.0,180.0,10000.0,10000.0,10000.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0,0
1270,1.0,1.0,31.0,2006.0,1.0,2.0,10000.0,10000.0,10000.0,10000.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0,0
1271,2.0,2.0,85.9,2013.0,0.0,0.0,10000.0,230.0,150.0,250.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0
1272,1.0,1.0,47.0,2012.0,0.0,0.0,10000.0,810.0,760.0,570.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [65]:
model[0].transformers_[0][1].steps[1][1]

StandardScaler()

In [ ]:
explainer = _explainer(task, target)
explaination = explainer.explain_instance(model[0].fit_transform(X_test)[0], pred_model)
# explaination.__dict__["domain_mapper"].__dict___to_csv_local(explaination)
_to_csv_local(explaination, 0)

In [ ]:
pd.concat([X_test[X_test.index == 0], df_x], axis = 1)

In [ ]:
explainer = _explainer(task, target)
sp_obj = submodular_pick.SubmodularPick(explainer, model[0].fit_transform(X_train), pred_model, num_features= len(all_columns), num_exps_desired = 5, sample_size=2)

In [ ]:
lime_obj = sp_obj
all_df = []
for i in range(0,2):
    cols = lime_obj.explanations[i].domain_mapper.feature_names
    values = lime_obj.explanations[i].domain_mapper.feature_values
    arr = np.array(values).reshape((1, len(list(lime_obj.explanations[i].local_exp.values())[0])))
    all_df.append(pd.DataFrame(arr , columns = cols))

In [ ]:
lime_obj.explanations[i].domain_mapper.feature_names

In [ ]:
# pd.concat(all_df_local, ignore_index = True).fillna(0).to_csv("/model_interpret/local.csv")

In [ ]:
save_to_csv(task, target)

In [ ]:
model['preprocessor'].transformers_[1][1].__dict__

In [ ]:
model["preprocessor"].transformers[0][2]

In [ ]:
model['preprocessor'].transformers_[0][2]

In [ ]:
all_columns = model["preprocessor"].transformers[0][2] +list(model['preprocessor'].transformers_[1][1].categories_[0]) + list(model['preprocessor'].transformers_[1][1].categories_[1])

In [ ]:
model['preprocessor'].transformers_[1][1].categories_

In [ ]:
load_transform(pipeline_name)

In [ ]:
def _explainer(task, target):
    feature_names = all_columns
    if task == "classification":
        class_names = y_train[target].unique()
        # Fit the Explainer on the training data set using the LimeTabularExplainer 
        explainer = LimeTabularExplainer(load_transform(pipeline_name).fit_transform(X_train), feature_names = feature_names, class_names = class_names, mode = task, categorical_features = check_type(X_train))
    else:
        explainer = LimeTabularExplainer(load_transform(pipeline_name).fit_transform(X_train), feature_names = feature_names, mode = task)
    return explainer

In [ ]:
_explainer(task, target)

In [ ]:
all_columns = []
for i in range(len(model["preprocessor"].transformers_)):
    if model["preprocessor"].transformers_[i][0] == "num":
        all_columns.extend(model["preprocessor"].transformers[i][2])
    else:
        for j in range(len(model['preprocessor'].transformers_[1][1].categories_)):
            all_columns.extend(list(model['preprocessor'].transformers_[1][1].categories_[j]))

In [ ]:
all_columns

In [ ]:
model["preprocessor"].transformers_

In [11]:
model[0].transformers_[0][1].inverse_transform(X_test.iloc[:,:79])

ValueError: could not convert string to float: 'no'

In [12]:
model[0].transformers_[0][1]

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [ ]:
inverse_transform(X_test_ja.iloc[:,:79])